<a href="https://colab.research.google.com/github/yl4732/TFA-Group-Project/blob/main/v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1tGcX2qzS2GoaN5zFeD5DVJxqDmoQdmMs7QncZwFCEqU/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_All code should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach._

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [ ]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=0f2a5f3bd02eac68a5b52bb1d75652c4bafb95384c4a354f6a40ba16224a4533
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [ ]:
# all import statements needed for the project, for example:

import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
from datetime import datetime


ERROR: unknown command "pandasql"


In [ ]:
# any general notebook setup, like log formatting

In [ ]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library
* [ ] Write at least one unit test that tests this distance calculation function. 
* Taxi data:
    * [ ] Use `requests`, BeautifulSoup (`bs4`), and `pandas` to programmatically download the required CSV files & load into memory.
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Remove trips that start and/or end outside the designated [coordinate box]
* Using the function that calculates the distance between two coordinates in kilometers, add a column to each `pandas` DataFrame of data that calculates the distance between pickup and dropoff locations for each trip.
* Weather data:
    * [ ] Download the data manually in the link provided in the project doc.
    * [ ] Load the data from your local computer (using `pandas`), then clean the data, including: 
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
        * You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 
* Take a look at the lecture notes from the `pandas` lecture for hints on helpful functionality

### Calculating distance
_Write some prose that tells the reader what you're about to do here._

In [ ]:
def calculate_distance(from_coord, to_coord):
  lat1, lon1 = from_coord
  lat2, lon2 = to_coord
  radius = 6371 # km

  dlat = math.radians(lat2-lat1)
  dlon = math.radians(lon2-lon1)
  a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  d = radius * c
  return d
#calculate_distance((40.776440	, -73.959862), (40.767906, -73.962837))

In [ ]:
def test_calculate_distance():
  assert calculate_distance((40.560445, -74.242330), (40.908524, -73.717047))==58.795089791900814

In [ ]:
from typing import AnyStr
def add_distance_column(dataframe):
  coordinate_distance=[]
  for index, row in dataframe.iterrows():
    from_coord=(row['pickup_latitude'],row['pickup_longitude'])
    to_coord=(row['dropoff_latitude'],row['dropoff_longitude'])
    coordinate_distance.append(calculate_distance(from_coord,to_coord))  
  dataframe= pd.concat([dataframe,pd.DataFrame(coordinate_distance,columns=['coordinate_distance'])], axis=1)
  print(len(coordinate_distance))
  return dataframe


### Processing Taxi Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from bs4 import BeautifulSoup
import re
def find_taxi_csv_urls(TAXI_URL):
    response = requests.get(TAXI_URL)
    if not response.status_code == 200:
        return None
    result_page = BeautifulSoup(response.content,'lxml')
    taxi_urls = list()
    all_taxi_urls=list()

    for year in result_page.find_all('div',{"id":re.compile('faq.*')}):

        for month in year.find_all('a', {"title": 'Yellow Taxi Trip Records'}):
            taxi_urls.append(month.get('href'))

    for i in taxi_urls:
      if i[-11:-4]>="2009-01" and i[-11:-4]<="2015-06":
        all_taxi_urls.append(i)   
    return all_taxi_urls




def get_and_clean_month_taxi_data(url):
    df = pd.read_csv(url)
    df.columns = df.columns.str.lower()
    words = ['time', 'lon', 'lat', 'tip', 'distance','total']
    df_new = pd.DataFrame()
    for i in words:
      col=df.loc[:,df.columns.str.contains(i)]
      df_new=pd.concat([df_new, col], axis=1)
    mapping = {df_new.columns[0]:'pickup_datetime', 
           df_new.columns[1]: 'dropoff_datetime', 
           df_new.columns[2]: 'pickup_longitude', 
           df_new.columns[3]: 'dropoff_longitude',
           df_new.columns[4]: 'pickup_latitude',
           df_new.columns[5]: 'dropoff_latitude',
           df_new.columns[6]: 'tip',
           df_new.columns[7]: 'distance',
           df_new.columns[8]: 'total_amount'}
    yellow_taxi = df_new.rename(columns=mapping)
    yellow_taxi_clean = yellow_taxi.dropna()
    filter = yellow_taxi_clean.loc[yellow_taxi_clean.pickup_longitude.between(-74.242330,-73.717047) & yellow_taxi_clean.dropoff_longitude.between(-74.242330,-73.717047) 
    & yellow_taxi_clean.pickup_latitude.between(40.560445,40.908524)& yellow_taxi_clean.dropoff_latitude.between(40.560445,40.908524)]
    sample= filter.sample(n=2564)
    sample.reset_index(drop=True, inplace=True)
    return sample

    

def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls(TAXI_URL)
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        re=add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        #save monthly cleaned data into drive
        from pathlib import Path
        #get the month of the cleaned data
        month = csv_url[-11:]
        #create path for the month
        filepath = Path(f'/content/drive/MyDrive/TFA_test/monthly_data/{month}') 
        #check whether there is an existing parent path:
        #if there is, save file under the path; if there isn't, create a new parent path
        filepath.parent.mkdir(parents=True, exist_ok=True)  
        re.to_csv(filepath, index=False)  
        all_taxi_dataframes.append(re)

    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    
    #save the aggregate cleaned data into drive
    filepath = Path('/content/drive/MyDrive/TFA_test/taxi.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    taxi_data.to_csv(filepath, index=False)  
    return taxi_data

In [ ]:
a=pd.read_csv('/content/drive/MyDrive/TFA_test/monthly_data/2015-01.csv')
print(a)

          pickup_datetime     dropoff_datetime  pickup_longitude  \
0     2015-01-04 11:45:30  2015-01-04 11:53:14        -73.967659   
1     2015-01-11 05:40:34  2015-01-11 06:04:40        -73.984016   
2     2015-01-15 14:53:55  2015-01-15 15:09:55        -73.947823   
3     2015-01-09 23:37:38  2015-01-09 23:51:00        -73.989609   
4     2015-01-13 20:16:03  2015-01-13 20:32:07        -73.986198   
...                   ...                  ...               ...   
2559  2015-01-03 20:35:33  2015-01-03 20:37:43        -73.982002   
2560  2015-01-24 23:57:18  2015-01-25 00:13:38        -74.003342   
2561  2015-01-18 16:49:45  2015-01-18 17:02:06        -73.974670   
2562  2015-01-08 23:03:43  2015-01-08 23:34:35        -73.983040   
2563  2015-01-04 15:57:16  2015-01-04 16:01:35        -73.985565   

      dropoff_longitude  pickup_latitude  dropoff_latitude    tip  distance  \
0            -73.965454        40.756008         40.766533   1.55      1.00   
1            -73.789909  

In [ ]:
a["pickup_datetime"] = pd.to_datetime(a["pickup_datetime"])
a["dayofweek"] = a["pickup_datetime"].dt.dayofweek
a["year"] = a["pickup_datetime"].dt.year
a["month"] = a["pickup_datetime"].dt.month
a["day"] = a["pickup_datetime"].dt.day
a["hour"] = a["pickup_datetime"].dt.hour
a=a.drop(columns=['distance'])
a

,pickup_datetime,dropoff_datetime,pickup_longitude,dropoff_longitude,pickup_latitude,dropoff_latitude,tip,total_amount,coordinate_distance,dayofweek,year,month,day,hour
0,2015-01-04 11:45:30,2015-01-04 11:53:14,-73.967659,-73.965454,40.756008,40.766533,1.55,9.35,1.184941,6,2015,1,4,11
1,2015-01-11 05:40:34,2015-01-11 06:04:40,-73.984016,-73.789909,40.743591,40.646961,8.00,66.13,19.576706,6,2015,1,11,5
2,2015-01-15 14:53:55,2015-01-15 15:09:55,-73.947823,-73.971748,40.770859,40.765907,1.00,13.30,2.088758,3,2015,1,15,14
3,2015-01-09 23:37:38,2015-01-09 23:51:00,-73.989609,-73.952293,40.762680,40.769730,3.12,16.42,3.238871,4,2015,1,9,23
4,2015-01-13 20:16:03,2015-01-13 20:32:07,-73.986198,-73.982346,40.726257,40.756344,2.85,17.15,3.361178,1,2015,1,13,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2559,2015-01-03 20:35:33,2015-01-03 20:37:43,-73.982002,-73.981346,40.767986,40.767494,1.00,5.80,0.077765,5,2015,1,3,20
2560,2015-01-24 23:57:18,2015-01-25 00:13:38,-74.003342,-73.978668,40.733135,40.744980,0.00,13.30,2.460882,5,2015,1,24,23
2561,2015-01-18 16:49:45,2015-01-18 17:02:06,-73.974670,-73.981972,40.756886,40.765900,1.00,10.80,1.175928,6,2015,1,18,16
2562,2015-01-08 23:03:43,2015-01-08 23:34:35,-73.983040,-73.879776,40.767727,40.701225,10.40,52.03,11.418538,3,2015,1,8,23


### Processing Uber Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
def load_and_clean_uber_data(csv_file):
  df = pd.read_csv(csv_file)
  df=df.drop(columns=['Unnamed: 0','key','passenger_count'])
  df = df.dropna(how='any')
  df = df.loc[df.pickup_longitude.between(-74.242330,-73.717047) & df.dropoff_longitude.between(-74.242330,-73.717047) 
    & df.pickup_latitude.between(40.560445,40.908524)& df.dropoff_latitude.between(40.560445,40.908524)]
  df.reset_index(drop=True, inplace=True)
  return df

def get_uber_data():
  uber_dataframe = load_and_clean_uber_data(UBER_CSV)
  df=add_distance_column(uber_dataframe)
  df["pickup_datetime"] = pd.to_datetime(df['pickup_datetime'])
  df["dayofweek"] = df["pickup_datetime"].dt.dayofweek
  df["year"] = df["pickup_datetime"].dt.year
  df["month"] = df["pickup_datetime"].dt.month
  df["day"] = df["pickup_datetime"].dt.day
  df["hour"] = df["pickup_datetime"].dt.hour
  return df
uber=get_uber_data()
uber

34931


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,coordinate_distance,dayofweek,year,month,day,hour
0,7.50,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1.683323,3,2015,5,7,19
1,7.70,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,2.457590,4,2009,7,17,20
2,12.90,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,5.036377,0,2009,8,24,21
3,5.30,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,1.661683,4,2009,6,26,8
4,16.00,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,4.475450,3,2014,8,28,17
...,...,...,...,...,...,...,...,...,...,...,...,...
34926,7.00,2013-07-22 22:58:00+00:00,-73.971210,40.751287,-73.977215,40.752325,0.518833,0,2013,7,22,22
34927,4.10,2009-03-17 16:40:34+00:00,-73.989167,40.762935,-73.991962,40.754702,0.945251,1,2009,3,17,16
34928,6.00,2014-06-12 20:03:00+00:00,-73.976287,40.760280,-73.983277,40.763827,0.708619,3,2014,6,12,20
34929,20.83,2013-04-09 10:21:38+00:00,-73.988428,40.670634,-74.013862,40.714818,5.360608,1,2013,4,9,10


### Processing Weather Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    we = pd.read_csv(csv_file)
    f1 = ['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']
    we_1 = pd.DataFrame()
    for i in f1:
        col1=we.loc[:,we.columns.str.contains(i)]
        we_1=pd.concat([we_1, col1], axis=1)
    hourly_clean= we_1.dropna()
    hourly_clean.reset_index(drop=True, inplace=True)
    hourly_clean["DATE"] = pd.to_datetime(hourly_clean['DATE'])
    hourly_clean["YEAR"] = hourly_clean["DATE"].dt.year
    hourly_clean["MONTH"] = hourly_clean["DATE"].dt.month
    hourly_clean["DAY"] = hourly_clean["DATE"].dt.day
    hourly_clean["HOUR"] = hourly_clean["DATE"].dt.hour
    return hourly_clean

def clean_month_weather_data_daily(csv_file):
    we = pd.read_csv(csv_file)
    f2 = ['DATE', 'DailySustainedWindSpeed']
    we_2 = pd.DataFrame()
    for i in f2:
        col2=we.loc[:,we.columns.str.contains(i)]
        we_2=pd.concat([we_2, col2], axis=1)
    daily_clean= we_2.dropna()
    daily_clean.reset_index(drop=True, inplace=True)
    daily_clean["DATE"] = pd.to_datetime(daily_clean['DATE'])
    daily_clean["YEAR"] = daily_clean["DATE"].dt.year
    daily_clean["MONTH"] = daily_clean["DATE"].dt.month
    daily_clean["DAY"] = daily_clean["DATE"].dt.day
    daily_clean["HOUR"] = daily_clean["DATE"].dt.hour
    return daily_clean
    

def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["2009_weather.csv", 
                         "2010_weather.csv", 
                         "2011_weather.csv",
                         "2012_weather.csv",
                         "2013_weather.csv",
                         "2014_weather.csv",
                         "2015_weather.csv"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data

In [ ]:
load_and_clean_weather_data() 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DtypeWarning: Columns (8,9,10,17) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (8,9,10,17) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory

(                     DATE HourlyPrecipitation  HourlyWindSpeed
 0     2009-01-02T12:34:00                   T             14.0
 1     2009-01-02T12:51:00                   T             11.0
 2     2009-01-02T13:05:00                   T              7.0
 3     2009-01-02T13:30:00                   T              9.0
 4     2009-01-02T13:46:00                   T             10.0
 ...                   ...                 ...              ...
 9102  2015-12-31T18:51:00                0.00              3.0
 9103  2015-12-31T19:51:00                0.00              6.0
 9104  2015-12-31T20:51:00                0.00             10.0
 9105  2015-12-31T22:51:00                0.00              7.0
 9106  2015-12-31T23:51:00                0.00              5.0
 
 [38260 rows x 3 columns],                     DATE  DailySustainedWindSpeed
 0    2012-07-31T23:59:00                     10.0
 1    2012-08-01T23:59:00                      9.0
 2    2012-08-02T23:59:00                     12.0


In [ ]:
import glob
import os
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('/content/drive/MyDrive/taxi_data_clean/', '20*.csv'))), ignore_index= True)
df.drop(columns = ['Unnamed: 0'])
df.to_csv('taxi_data_clean.csv')
#df['tip'].isna().sum()

### Process All Data

_This is where you can actually execute all the required functions._

_Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = pd.read_csv('taxi_data_clean.csv')
uber_data = get_uber_data()
hourly_weather, daily_weather = load_and_clean_weather_data()
all_trips = pd.concat([taxi_data, uber_data], axis=0, ignore_index=True)
all_trips=all_trips.drop(columns=['dropoff_datetime','tip','total_amount','fare_amount'])


195472


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/

In [ ]:
a["pickup_datetime"] = pd.to_datetime(a["pickup_datetime"])
a["dayofweek"] = a["pickup_datetime"].dt.dayofweek
a["year"] = a["pickup_datetime"].dt.year
a["month"] = a["pickup_datetime"].dt.month
a["day"] = a["pickup_datetime"].dt.day
a["hour"] = a["pickup_datetime"].dt.hour
a=a.drop(columns=['distance'])
a

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    DATE TEXT,
    HourlyPrecipitation FLOAT,
    HourlyWindSpeed FLOAT,
    YEAR INTEGER,
    MONTH INTEGER,
    DAY INTEGER,
    HOUR INTEGER
)
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    DATE TEXT,
    DailySustainedWindSpeed FLOAT,
    YEAR INTEGER,
    MONTH INTEGER,
    DAY INTEGER,
    HOUR INTEGER
)
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    pickup_longitude FLOAT,
    dropoff_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_latitude FLOAT,
    tip FLOAT,
    total_amount FLOAT,
    coordinate_distance FLOAT,
    dayofweek INTEGER,
    year INTEGER,
    month INTEGER,
    day INTEGER,
    hour INTEGER


)
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    fare_amount FLOAT,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude	FLOAT,
    dropoff_latitude FLOAT,
    coordinate_distance FLOAT,
    dayofweek INTEGER,
    year INTEGER,
    month INTEGER,
    day INTEGER,
    hour INTEGER

)
"""

ALL_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS all_trips
(
    id INTEGER PRIMARY KEY,
    pickup_datetime DATETIME,
    pickup_longitude FLOAT,
    dropoff_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_latitude FLOAT,
    coordinate_distance FLOAT,
    dayofweek INTEGER,
    year INTEGER,
    month INTEGER,
    day INTEGER,
    hour INTEGER

)
"""


In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)
    f.write(ALL_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_Write some prose that tells the reader what you're about to do here._

In [ ]:
def dataframes_to_table(table_name_to_dataframe):
  for key in table_name_to_dataframe:
    table_name_to_dataframe[key].to_sql(key, con=engine, if_exists="replace", index=False)

In [ ]:
table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather,
    "daily_weather": daily_weather,
    "all_trips":all_trips
}

dataframes_to_table(table_name_to_dataframe)

In [ ]:
#engine.execute("SELECT * FROM daily_weather").fetchall()

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
#Q1
engine.execute("SELECT hour AS Hour, COUNT(*) AS Trips FROM taxi_trips GROUP BY Hour ORDER BY Trips DESC").fetchall()


In [ ]:
#Q2
q2 = engine.execute("SELECT dayofweek AS Weekday, COUNT(*) AS Trips FROM uber_trips GROUP BY dayofweek ORDER BY Trips DESC").fetchall()
write_query_to_file(q2, 1)

In [ ]:
#Q3
query = """
SELECT coordinate_distance AS '95% percentile of distance'
FROM all_trips
WHERE year = 2013 AND month = 7
ORDER BY coordinate_distance ASC
LIMIT 1
OFFSET (
SELECT CAST(COUNT(coordinate_distance)*0.95-1 AS INT) FROM all_trips WHERE year = 2013 AND month = 7)
"""
engine.execute(query).fetchall()

[(10.231468318625527,)]

In [ ]:
#Q4
query = """
SELECT month, day, COUNT(*) AS trips, AVG(coordinate_distance) as average_distance
FROM all_trips
WHERE year = 2009
GROUP BY month, day
ORDER BY trips DESC
LIMIT 10
"""
engine.execute(query).fetchall()

[(12, 11, 127, 2.97589203472042),
 (10, 23, 123, 2.4524671315098834),
 (8, 14, 119, 3.548617620635577),
 (4, 18, 113, 3.329794125272145),
 (1, 31, 112, 2.7919509511955884),
 (5, 8, 111, 3.3112073580725196),
 (7, 9, 111, 3.4544163807019315),
 (3, 19, 110, 3.294181937318452),
 (5, 16, 109, 2.7727537127507222),
 (4, 4, 108, 2.6060268703851786)]

In [ ]:
#Q5
query = """
SELECT mix.month, mix.day, DailySustainedWindSpeed as WindSpeed, COUNT(*) as trips
FROM daily_weather mix
JOIN all_trips daily_weather
ON mix.month = daily_weather.month AND mix.day = daily_weather.day AND mix.year = daily_weather.YEAR 
WHERE mix.year = 2014
GROUP BY mix.month, mix.day
ORDER BY DailySustainedWindSpeed DESC
LIMIT 10
"""
engine.execute(query).fetchall()

[(2, 13, 25.0, 63),
 (12, 7, 25.0, 73),
 (1, 7, 23.0, 77),
 (3, 13, 23.0, 106),
 (3, 26, 23.0, 96),
 (3, 29, 23.0, 107),
 (1, 22, 22.0, 61),
 (2, 14, 22.0, 71),
 (11, 18, 22.0, 86),
 (12, 9, 22.0, 70)]

In [ ]:
#Q6
query = """
SELECT re.month, re.day, re.hour, HourlyPrecipitation, HourlyWindSpeed, COUNT(*) as trips
FROM hourly_weather re
JOIN all_trips hourly_weather
ON re.year = hourly_weather.year AND re.month = hourly_weather.month AND re.day = hourly_weather.day AND re.hour = hourly_weather.hour
WHERE re.year = 2012 AND ((re.month = 10 AND re.day >= 22) OR (re.month = 11 AND re.day <= 6))
GROUP BY re.month, re.day, re.hour
"""
engine.execute(query).fetchall()

[(10, 22, 0, '0.00', 7.0, 2),
 (10, 22, 2, '0.00', 7.0, 1),
 (10, 22, 3, '0.00', 0.0, 1),
 (10, 22, 5, '0.00', 0.0, 1),
 (10, 22, 6, '0.00', 5.0, 3),
 (10, 22, 7, '0.00', 3.0, 5),
 (10, 22, 8, '0.00', 3.0, 1),
 (10, 22, 9, '0.00', 5.0, 5),
 (10, 22, 12, '0.00', 11.0, 5),
 (10, 22, 14, '0.00', 7.0, 1),
 (10, 22, 15, '0.00', 6.0, 1),
 (10, 22, 16, '0.00', 3.0, 6),
 (10, 22, 17, '0.00', 7.0, 4),
 (10, 22, 18, '0.00', 5.0, 8),
 (10, 22, 19, '0.00', 5.0, 2),
 (10, 22, 20, '0.00', 3.0, 4),
 (10, 22, 21, '0.00', 0.0, 1),
 (10, 22, 22, '0.00', 3.0, 6),
 (10, 23, 12, 'T', 0.0, 1),
 (10, 23, 19, 'T', 0.0, 2),
 (10, 23, 20, '0.02', 0.0, 8),
 (10, 23, 21, 'T', 5.0, 4),
 (10, 23, 22, '0.01', 0.0, 6),
 (10, 23, 23, 'T', 5.0, 3),
 (10, 24, 2, '0.00', 5.0, 4),
 (10, 24, 5, '0.00', 6.0, 2),
 (10, 24, 7, 'T', 0.0, 1),
 (10, 24, 8, 'T', 0.0, 2),
 (10, 24, 9, 'T', 3.0, 4),
 (10, 24, 10, '0.00', 7.0, 2),
 (10, 24, 11, '0.00', 7.0, 4),
 (10, 24, 12, 'T', 7.0, 6),
 (10, 24, 13, '0.00', 8.0, 4),
 (10, 24, 14,

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.
* [ ] Come up with 3 questions on your own that can be answered based on the data in the 4 tables. Create at least one visualization to answer each question. At least one visualization should require data from at least 3 tables.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_You don't have to query the data directly from the database. You can just re-use the pandas DataFrame that you created in Part 1._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
plot_visual_n(some_dataframe)